In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('C:/users/LL/Documents/GitHub/AMLF_projects/data.csv')

In [3]:
df = df.drop(columns = ['Unnamed: 0'])

In [4]:
df.head()

,Date,permno,return,mom1m,mom12m,chmom,indmom,mom36m,turn,mvel1,...,baspread,retvol,idiovol,beta,betasq,ep,sp,agr,nincr,return(t-1)
0,2001-01-31,13610,-0.202242,0.277978,-0.082232,0.518490,0.198455,-0.258322,0.820391,9.897840e+05,...,0.037278,0.032888,0.056769,0.398706,0.158967,0.019041,1.472909,0.325935,5.0,NaN
1,2001-01-31,13856,-0.112756,0.095372,0.300233,-0.147620,0.069669,0.000718,0.660636,7.152231e+07,...,0.033983,0.022389,0.042020,0.106574,0.011358,0.039970,0.397105,0.225463,1.0,NaN
2,2001-01-31,13901,0.010566,0.166088,0.759861,0.504130,0.400659,-0.440929,0.775189,9.783550e+07,...,0.032844,0.023475,0.050243,0.088382,0.007811,0.142695,1.148088,-0.024383,2.0,NaN
3,2001-01-31,13928,0.033114,0.006637,0.236486,0.039925,0.400659,0.025686,0.813903,1.451888e+07,...,0.035964,0.023917,0.037427,0.159983,0.025595,0.051091,1.138525,-0.069288,1.0,NaN
4,2001-01-31,13936,0.014335,0.009709,0.574141,0.224657,-0.075486,-0.397110,0.755288,3.550430e+05,...,0.031413,0.030343,0.068491,1.060490,1.124640,0.091598,6.902488,0.000838,0.0,NaN


In [5]:
print(len(df.columns))
print(df.columns)

23
Index(['Date', 'permno', 'return', 'mom1m', 'mom12m', 'chmom', 'indmom',
       'mom36m', 'turn', 'mvel1', 'dolvol', 'ill', 'zerotrade', 'baspread',
       'retvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr',
       'return(t-1)'],
      dtype='object')


In [6]:
df['dolvol']

0         12.968131
1         17.527472
2         17.773933
3         16.111030
4         12.518458
            ...    
113995    10.203254
113996     9.558279
113997    11.116588
113998    14.259460
113999    14.433653
Name: dolvol, Length: 114000, dtype: float64

In [7]:
df.dtypes

Date            object
permno           int64
return         float64
mom1m          float64
mom12m         float64
chmom          float64
indmom         float64
mom36m         float64
turn           float64
mvel1          float64
dolvol         float64
ill            float64
zerotrade      float64
baspread       float64
retvol         float64
idiovol        float64
beta           float64
betasq         float64
ep             float64
sp             float64
agr            float64
nincr          float64
return(t-1)    float64
dtype: object

According to note 30: "Therefore, to predict returns at month t+1, we use most recent monthly characteristics at the end of month t." <br>
Hence, **shift return t+1 to serve as response: r(t+1)**.

In [8]:
df['r(t+1)'] = df.groupby('permno')['return'].shift(-1)

In [9]:
len(df)

114000

In [10]:
df.isna().sum()

Date              0
permno            0
return            0
mom1m             3
mom12m          269
chmom           269
indmom            0
mom36m         1566
turn              9
mvel1             0
dolvol            6
ill               0
zerotrade         0
baspread          0
retvol            0
idiovol         352
beta            352
betasq          352
ep              473
sp              473
agr            1026
nincr          2794
return(t-1)     500
r(t+1)          500
dtype: int64

### handle missing data

According to note 30 (bottom of p 2248): "Another issue is missing characteristics, which we replace with the cross-sectional median at each month for each stock, respectively." <br>
Hence, calculate monthly cross-sectional median for features: **'mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr'**.

In [11]:
df_filled = df.copy()
for feature in ['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']:
    df_filled[feature] = df_filled.groupby('Date')[feature].transform(lambda x: x.fillna(x.median()))

In [12]:
df_filled.isna().sum()

Date             0
permno           0
return           0
mom1m            0
mom12m           0
chmom            0
indmom           0
mom36m           0
turn             0
mvel1            0
dolvol           0
ill              0
zerotrade        0
baspread         0
retvol           0
idiovol          0
beta             0
betasq           0
ep               0
sp               0
agr              0
nincr            0
return(t-1)    500
r(t+1)         500
dtype: int64

In [13]:
df.loc[:, ['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']] = df_filled.loc[:,['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']]

In [14]:
df.isna().sum()

Date             0
permno           0
return           0
mom1m            0
mom12m           0
chmom            0
indmom           0
mom36m           0
turn             0
mvel1            0
dolvol           0
ill              0
zerotrade        0
baspread         0
retvol           0
idiovol          0
beta             0
betasq           0
ep               0
sp               0
agr              0
nincr            0
return(t-1)    500
r(t+1)         500
dtype: int64

In [15]:
df['Date'] = pd.to_datetime(df['Date'])

# Set the datetime column as index
df.set_index('Date', inplace=True, drop = True)

In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df_scaled = scaler.fit_transform(df)

df_scaled = pd.DataFrame(df_scaled, columns=df.columns)


In [17]:
permno = df['permno'].reset_index(drop = True)

In [18]:
df_scaled['permno'] = permno

In [19]:
df_scaled.index = df.index

In [20]:
df_scaled_2 = df_scaled.drop(columns = [ 'permno', 'return'])

### split data

**split training, validation, and testing datasets**

training : validation : testing = 6 yr : 4yr : 9 yr <br>
Also drop the first and last month due to the absence of r(t+1) and return(t-1)

In [21]:
# training = df_scaled[:'2007-01-01'].dropna()
# validation = df_scaled['2007-01-01':'2011-01-01']
# testing = df_scaled['2011-01-01':].dropna()

In [22]:
# training_combined = df_scaled[:'2011-01-01'].dropna()

**separate X and y**

In [23]:
# X_train = training.drop(columns = ['r(t+1)'])
# y_train = training['r(t+1)']

In [24]:
# X_val = validation.drop(columns = ['r(t+1)'])
# y_val = validation['r(t+1)']

In [25]:
# X_test = testing.drop(columns = ['r(t+1)'])
# y_test = testing['r(t+1)']

In [26]:
# X_train_combined = training_combined.drop(columns = ['r(t+1)'])
# y_train_combined = training_combined['r(t+1)']

In [27]:
# print(X_train.isna().values.sum())
# print(X_val.isna().values.sum())
# print(X_test.isna().values.sum())
# print(y_train.isna().values.sum())
# print(y_val.isna().values.sum())
# print(y_test.isna().values.sum())

In [28]:
# print(training.shape)
# print(validation.shape)
# print(testing.shape)

### generate rolling window

In [29]:
def generate_rolling_windows(data, total_years, initial_train_size, val_size):
    total_years = total_years
    windows = []
    initial_train_size = initial_train_size
    val_size = val_size

    # Loop through the years to create rolling windows
    for i in range(initial_train_size + 1, total_years-val_size):
        idx_1 = '20' + str(i).zfill(2) + '-01-01'
        idx_2 = '20' + str(i + val_size).zfill(2) + '-01-01'
        
        training = data[:idx_1].dropna()
        validation = data[idx_1:idx_2]
        testing = data[idx_2:].dropna()
        
        train_com = data[:idx_2].dropna()

        windows.append((training, validation, testing, train_com))
        
    return windows


# for inital split: training, validation, testing, train_combined 
#                       - windows[0][0], windows[0][1], windows[0][2], windows[0][3]

# increase the first index by 1 using for loop to fit all data

In [30]:
# generate rolling samples

windows = generate_rolling_windows(df_scaled_2, 19, 6, 4)

In [31]:
len(windows)

8

### subsetting data: top and bottom 100 stocks on market value (mvel1)

In [32]:
subset = df_scaled.copy()

In [33]:
mvel_sorted_top = subset['2019-12-01':].sort_values('mvel1',ascending=False).head(100).reset_index(drop=True)
filter_values_top = mvel_sorted_top['permno']
top_100 = subset[subset['permno'].isin(filter_values_top)]


mvel_sorted_bot = subset['2019-12-01':].sort_values('mvel1',ascending=False).tail(100).reset_index(drop=True)
filter_values_bot = mvel_sorted_bot['permno']
bot_100 = subset[subset['permno'].isin(filter_values_bot)]


In [34]:
top_100.shape
# 19*12*10 = 2280

(22800, 23)

**splitting data & separating X and y** <br>
***
splitting data 10:9 years - to make the training set the same size as the training and validation set combined, leave testing set out of sample

In [35]:
training_top = top_100[:'2011-01-01'].dropna()
testing_top = top_100['2011-01-02':].dropna()

In [36]:
training_bot = bot_100[:'2011-01-01'].dropna()
testing_bot = bot_100['2011-01-02':].dropna()

separating X and y

In [37]:
X_test_top = testing_top.drop(columns = ['permno', 'return', 'r(t+1)'])
y_test_top = testing_top['r(t+1)']

X_test_bot = testing_bot.drop(columns = ['permno', 'return', 'r(t+1)'])
y_test_bot = testing_bot['r(t+1)']

## packages & custom funcs
***
1. R^2 without demeaning
2. tuning_1: fixed set hyperparameter tuning
3. tuning: recursive rolling tuning

In [38]:
from sklearn.metrics import r2_score

In [39]:
# r^2-oos for Enet

def r2_score_wo_demeaning(y_true, y_pred):
    y_true = np.array(y_true)
    
    ss_res = 0
    ss_tot = 0
    for i in range(len(y_true)):
        ss_res = ss_res + ((y_true[i] - y_pred[i])**2)
        ss_tot = ss_tot + ((y_true[i] - 0)**2)
    r2 = 1 - (ss_res / ss_tot)
    return float(r2)

In [40]:
def tuning_1 (model, param_combos, X_train, y_train, X_val, y_val):
#   param_combos: list of dictionaries of parameters combo
    
    results = {'param': [], 'score': []}
    opt_param = 0
    opt_score = 0

    for params in param_combos:

        for param_name, param_value in params.items():
            setattr(model, param_name, param_value)

        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)

#         print("Shape of y_val:", y_val.shape)
#         print("Shape of y_pred:", y_pred.shape)
        
        r2 = r2_score_wo_demeaning(y_val, y_pred)

        results['param'].append(params)
        results['score'].append(r2)
    
    sorted_zipped = sorted(zip(results['param'], results['score']), key=lambda x: x[1])
    opt_param = sorted_zipped[0][0]
    opt_score = sorted_zipped[0][1]
    
    return opt_param, opt_score

## Enet


prediction: g*(z i,t) - depends on neither i nor t, is dependend on z only through z i,t <br>
responses: r i, t+1


In [42]:
from sklearn.linear_model import ElasticNet

**tuning**

1. set-up the search grid
2. tuning: recursive rolling validation set w/ oos R^2 metric -- <br>
    roll one unit froward each time and take the average as the final score <br>
3. pick the best hyperparameters, retrain the model

In [43]:
# set up searching grid

alpha_values = [0.4, 0.5, 0.6]
l1_ratio_values = np.logspace(-4, -1, 10)

param_enet = [{'alpha': alpha, 'l1_ratio': l1_ratio} for alpha in alpha_values for l1_ratio in l1_ratio_values]

# for param_combination in param_combinations:
#     print(param_combination)

In [44]:
# the model

Enet = ElasticNet()

In [45]:
results = []
for window in windows:
    
    #apply tuning function
    result = tuning_1(Enet, param_enet, window[0].drop(columns = ['r(t+1)']), window[0]['r(t+1)'], window[1].drop(columns = ['r(t+1)']), window[1]['r(t+1)'])
    
    # print out tuning parameters of eah]ch split
    print(f"the optimal parameters are {result[0]} and the best score is {result[1]}.")
    
    results.append(result)

the optimal parameters are {'alpha': 0.6, 'l1_ratio': 0.1} and the best score is 0.0007755796797865866.
the optimal parameters are {'alpha': 0.6, 'l1_ratio': 0.1} and the best score is 0.002077004617268674.
the optimal parameters are {'alpha': 0.6, 'l1_ratio': 0.1} and the best score is -0.0014524607777255394.
the optimal parameters are {'alpha': 0.4, 'l1_ratio': 0.0001} and the best score is -0.0063834100482236256.
the optimal parameters are {'alpha': 0.4, 'l1_ratio': 0.0001} and the best score is -0.004957498963627627.
the optimal parameters are {'alpha': 0.4, 'l1_ratio': 0.0001} and the best score is -0.0035499187056395876.
the optimal parameters are {'alpha': 0.4, 'l1_ratio': 0.0001} and the best score is -0.0012202180037492738.
the optimal parameters are {'alpha': 0.4, 'l1_ratio': 0.0001} and the best score is -0.0022601722569968175.


In [46]:
# for window in windows:
#     print(window[0].drop(columns = ['r(t+1)']))

**best model** <br>
<br>
Fit on all data in training and validation sets.

In [54]:
Enet_opt = ElasticNet()
r2_Enet = []
r2_Enet_top = []
r2_Enet_bot = []
for i in range(8):
    
    for param_name, param_value in results[i][0].items():
     setattr(Enet_opt, param_name, param_value)
    
    Enet_opt.fit(windows[i][3].drop(columns = ['r(t+1)']), windows[i][3]['r(t+1)'])
    y_pred_enet = Enet_opt.predict(windows[i][2].drop(columns = ['r(t+1)']))
    
    r2_enet = r2_score_wo_demeaning(windows[i][2]['r(t+1)'], y_pred_enet)
    r2_Enet.append(r2_enet)       # fit best model for each split
    
    # used model to generate prediction for the top 100
    y_pred_enet_top = Enet_opt.predict(X_test_top)
    r2_enet_top = r2_score_wo_demeaning(y_test_top, y_pred_enet_top)
    r2_Enet_top.append(r2_enet_top)
    
    y_pred_enet_bot = Enet_opt.predict(X_test_bot)
    r2_enet_bot = r2_score_wo_demeaning(y_test_bot, y_pred_enet_bot)
    r2_Enet_bot.append(r2_enet_bot)

In [55]:
# all the best r^2-oos for 8 splits stored in one list:

r2_Enet

[0.0005663579684225262,
 0.0007243052297416508,
 0.00015885305985385845,
 -0.001960710816258926,
 -0.0008903639645345685,
 2.6038543099682343e-05,
 -0.002316165787667135,
 -0.00016025654555895663]

In [56]:
#take average

print("R^2-oos:", sum(r2_Enet) / len(r2_Enet))

R^2-oos: -0.0004814927891127335


**top and bottom 100 prediction**

In [57]:
r2_Enet_top

[0.0026816164329585,
 0.0032536355833986397,
 0.0033384827679580065,
 -0.014004771665988347,
 -0.013416054442168646,
 -0.017310167694035927,
 -0.011820903699103269,
 -0.008782282295214605]

In [58]:
# take average

print("Top 100 stocks R^2-oos:", sum(r2_Enet_top) / len(r2_Enet_top))

Top 100 stocks R^2-oos: -0.007007555626524456


In [59]:
r2_Enet_bot

[-0.002231535569684029,
 -0.0009302732914853173,
 -0.0010422973876647035,
 -0.005407781758678176,
 -0.004479592866621029,
 -0.002450936187006425,
 -0.0032335221090211697,
 -0.0031839051781878958]

In [60]:
# take average

print("Bottom 100 stocks R^2-oos:", sum(r2_Enet_bot) / len(r2_Enet_bot))

Bottom 100 stocks R^2-oos: -0.0028699805435435932


**feature importance**

In [ ]:
# import matplotlib.pyplot as plt

# coefficients = Enet_opt.coef_

# # Get feature names
# # Assuming feature_names is a list of your feature names
# feature_names = X_train_combined.columns  # Insert your feature names here

# # Create a dictionary to store feature importance scores with feature names
# feature_importance_dict = dict(zip(feature_names, np.abs(coefficients)))

# # Sort feature importance dictionary by importance score
# sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# # Extract feature names and importance scores after sorting
# sorted_feature_names = [x[0] for x in sorted_feature_importance]
# sorted_feature_importance_scores = [x[1] for x in sorted_feature_importance]

# # Plotting
# plt.figure(figsize=(8, 6))
# plt.barh(sorted_feature_names, sorted_feature_importance_scores)
# plt.xlabel('Feature Importance')
# plt.ylabel('Features')
# plt.title('Feature Importance of Elastic Net Model')
# plt.gca().invert_yaxis()  # Invert y-axis to have the most important features at the top
# plt.show()

## NN

In [41]:
import keras
from keras import layers
from keras import Sequential

**loss function**

Customize the loss function to be **"penalized cross-sectional average prediction error" (appendix B.3) aggregated over time**, which is the same as **penalized l2 objective function of prediction errors (p2244)**. <br>
***
penalty term:

In [42]:
def custom_loss_pcape (y_true, y_pred):
    return

**performance metric**

Out-of-sample R^2. (r2_score_wo_demeaning)

In [42]:
import tensorflow as tf

def r2_score_wo_demeaning_nn(y_true, y_pred):
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
    ss_tot = tf.reduce_sum(tf.square(y_true - 0))
    r2 = 1 - (ss_res / ss_tot)
    return r2

**tuning custom function for neural net**

In [43]:

def compile_and_tune_model(model, parameter_dicts, x_train, y_train, x_val, y_val):
    results = []
    
    for params in parameter_dicts:

        model.compile(optimizer=keras.optimizers.Adam(learning_rate=params['learning_rate']),
                      loss='mean_squared_error', metrics=r2_score_wo_demeaning_nn)  # Using mean absolute error (mae) as metric
        
        history = model.fit(x_train, y_train, epochs=params['epoch'], batch_size=params['batch_size'],
                            validation_data=(x_val, y_val), verbose=0)
        
        # Get the metric value for the last epoch
        last_epoch_metric = history.history['r2_score_wo_demeaning_nn'][-1]  # Validation MAE for last epoch
        
        # Store results for current parameter set
        results.append({'params': params, 'val_r2_score_wo_demeaning_nn': last_epoch_metric})
    
    return results

### model

All activation functions are ReLU function <br>
optimizer: SGD w/ learning rate shrinkage: adam <br>

convert dataframes to numpy array

set-up grid

In [44]:
nepoch_val = [25, 50, 75, 100]
lr_val = [0.05, 0.025, 0.01]
nbatch_val = [1000, 1500, 3000]

param_nn = [{'epoch': epoch, 'learning_rate': learning_rate, 'batch_size': batch_size} for epoch in nepoch_val for learning_rate in lr_val for batch_size in nbatch_val]

**1-layer** <br>

In [46]:
input_dim = 20
layer1_n = 32

model_1 = Sequential([
            layers.Dense(layer1_n, activation='relu', input_dim=input_dim),
            layers.Dense(1, activation='linear')
        ])

tuning

In [49]:
results_nn1 = []
opt_scores_nn1 = []
opt_paras_nn1 = []

for window in windows:
    print("start running")
    # to arry
    X_train_nn = np.asarray(window[0].drop(columns = ['r(t+1)']).values)
    y_train_nn = np.asarray(window[0]['r(t+1)'].values)

    X_val_nn = np.asarray(window[1].drop(columns = ['r(t+1)']).values)
    y_val_nn = np.asarray(window[1]['r(t+1)'].values)
    
    # apply function
    result_nn1 = compile_and_tune_model(model_1, param_nn, X_train_nn, y_train_nn, X_val_nn, y_val_nn)
    
    # get score
    result_nn1 = pd.DataFrame(result_nn1).sort_values('val_r2_score_wo_demeaning_nn',ascending=False)
    print('finished')
    opt_score = result_nn1.iloc[0,1]
    print("opt_score: ", opt_score)
    opt_para = result_nn1.iloc[0,0]
    print("opt_result: ", opt_para)
    # results_nn1.append(result_nn1.iloc[0,])
    opt_scores_nn1.append(opt_score)
    opt_paras_nn1.append(opt_para)

start running
finished
opt_score:  0.030491596087813377
opt_result:  {'epoch': 75, 'learning_rate': 0.01, 'batch_size': 3000}
start running
finished
opt_score:  0.02640530839562416
opt_result:  {'epoch': 25, 'learning_rate': 0.01, 'batch_size': 3000}
start running
finished
opt_score:  0.021786686033010483
opt_result:  {'epoch': 100, 'learning_rate': 0.01, 'batch_size': 3000}
start running
finished
opt_score:  0.04627640172839165
opt_result:  {'epoch': 25, 'learning_rate': 0.01, 'batch_size': 3000}
start running
finished
opt_score:  0.041678592562675476
opt_result:  {'epoch': 50, 'learning_rate': 0.01, 'batch_size': 3000}
start running
finished
opt_score:  0.03884220868349075
opt_result:  {'epoch': 50, 'learning_rate': 0.01, 'batch_size': 3000}
start running
finished
opt_score:  0.03660942241549492
opt_result:  {'epoch': 25, 'learning_rate': 0.01, 'batch_size': 3000}
start running
finished
opt_score:  0.0348481610417366
opt_result:  {'epoch': 25, 'learning_rate': 0.01, 'batch_size': 300

In [50]:
result_nn1

,params,val_r2_score_wo_demeaning_nn
8,"{'epoch': 25, 'learning_rate': 0.01, 'batch_si...",0.034848
17,"{'epoch': 50, 'learning_rate': 0.01, 'batch_si...",0.034680
35,"{'epoch': 100, 'learning_rate': 0.01, 'batch_s...",0.034348
26,"{'epoch': 75, 'learning_rate': 0.01, 'batch_si...",0.034133
23,"{'epoch': 75, 'learning_rate': 0.025, 'batch_s...",0.033955
34,"{'epoch': 100, 'learning_rate': 0.01, 'batch_s...",0.033724
16,"{'epoch': 50, 'learning_rate': 0.01, 'batch_si...",0.033336
7,"{'epoch': 25, 'learning_rate': 0.01, 'batch_si...",0.033214
25,"{'epoch': 75, 'learning_rate': 0.01, 'batch_si...",0.033107
32,"{'epoch': 100, 'learning_rate': 0.025, 'batch_...",0.032542


best model

In [ ]:
# result_nn1.iloc[0,0]

In [ ]:
# result_nn1.iloc[0,1]

In [52]:
model_nn1 = Sequential([
            layers.Dense(layer1_n, activation='relu', input_dim=input_dim),
            layers.Dense(1, activation='linear')
        ])

last_epoch_metrics = []
r2_nn1_tops = []
r2_nn1_bots = []

for i in range(8):
    
    model_nn1.compile(optimizer=keras.optimizers.Adam(learning_rate=opt_para['learning_rate']),
                          loss='mean_squared_error', metrics=r2_score_wo_demeaning_nn)

    X_train_combined_nn = np.asarray(windows[i][3].drop(columns = ['r(t+1)']).values)
    y_train_combined_nn = np.asarray(windows[i][3]['r(t+1)'].values)
    
    X_test_nn = np.asarray(windows[i][2].drop(columns = ['r(t+1)']).values)
    y_test_nn = np.asarray(windows[i][2]['r(t+1)'].values)
    
    # fit the model
    print('start fitting')
    history = model_nn1.fit(X_train_combined_nn, y_train_combined_nn, epochs=opt_paras_nn1[i]['epoch'], batch_size=opt_paras_nn1[i]['batch_size'],
                                validation_data=(X_test_nn, y_test_nn), verbose=0)
    print('finished fitting')
    
    # get scores
    last_epoch_metric = history.history['val_r2_score_wo_demeaning_nn'][-1]
    last_epoch_metrics.append(last_epoch_metric)
    print('best score: ', last_epoch_metric)
    
    # top 100
    y_pred_nn1_top = model_nn1.predict(X_test_top)
    r2_nn1_top = r2_score_wo_demeaning(y_test_top, y_pred_nn1_top)
    r2_nn1_tops.append(r2_nn1_top)
    print('top best score: ', r2_nn1_top)
    
    # bot 100
    y_pred_nn1_bot = model_nn1.predict(X_test_bot)
    r2_nn1_bot = r2_score_wo_demeaning(y_test_bot, y_pred_nn1_bot)
    r2_nn1_bots.append(r2_nn1_bot)
    print('bot best score: ', r2_nn1_bot)

start fitting
finished fitting
best score:  -0.03335870802402496
335/335 [==============================] - 1s 2ms/step
top best score:  -0.09111201763153076
335/335 [==============================] - 1s 2ms/step
bot best score:  -0.026174426078796387
start fitting
finished fitting
best score:  -0.031564392149448395
335/335 [==============================] - 1s 2ms/step
top best score:  -0.07620644569396973
335/335 [==============================] - 1s 2ms/step
bot best score:  -0.015497922897338867
start fitting
finished fitting
best score:  -0.03095974214375019
335/335 [==============================] - 1s 2ms/step
top best score:  -0.08349168300628662
335/335 [==============================] - 1s 2ms/step
bot best score:  -0.013969063758850098
start fitting
finished fitting
best score:  -0.031458716839551926
335/335 [==============================] - 1s 2ms/step
top best score:  -0.03973591327667236
335/335 [==============================] - 1s 2ms/step
bot best score:  -0.015452504

In [53]:
last_epoch_metrics

[-0.03335870802402496,
 -0.031564392149448395,
 -0.03095974214375019,
 -0.031458716839551926,
 -0.04084852337837219,
 -0.03478659689426422,
 -0.01993107795715332,
 -0.03461456298828125]

In [54]:
print("NN1 average R^2-oos:", sum(last_epoch_metrics) / len(last_epoch_metrics))

NN1 average R^2-oos: -0.03219029004685581


**top and bottom 100 prediction**

In [55]:
r2_nn1_tops

[-0.09111201763153076,
 -0.07620644569396973,
 -0.08349168300628662,
 -0.03973591327667236,
 -0.029665589332580566,
 -0.09984898567199707,
 -0.011535048484802246,
 -0.0009429454803466797]

In [56]:
r2_nn1_bots

[-0.026174426078796387,
 -0.015497922897338867,
 -0.013969063758850098,
 -0.01545250415802002,
 -0.02376711368560791,
 -0.008506417274475098,
 0.00767594575881958,
 -0.0010194778442382812]

In [57]:
print("Top 100 stocks R^2-oos:", sum(r2_nn1_tops) / len(r2_nn1_tops))

Top 100 stocks R^2-oos: -0.054067328572273254


In [58]:
print("Bottom 100 stocks R^2-oos:", sum(r2_nn1_bots) / len(r2_nn1_bots))

Bottom 100 stocks R^2-oos: -0.012088872492313385


**3-layer**

In [45]:
input_dim = 20
layer1_n = 32
layer2_n = 16
layer3_n = 8


model_3 = Sequential([
            layers.Dense(layer1_n, input_dim = input_dim, activation='relu'),
            layers.Dense(layer2_n, activation='relu'),
            layers.Dense(layer3_n, activation='relu'),
            layers.Dense(1, activation='linear')
        ])


tuning

In [51]:
results_nn3 = []
opt_scores_nn3 = []
opt_paras_nn3 = []

for window in windows:
    print('start running')
    # to arry
    X_train_nn = np.asarray(window[0].drop(columns = ['r(t+1)']).values)
    y_train_nn = np.asarray(window[0]['r(t+1)'].values)

    X_val_nn = np.asarray(window[1].drop(columns = ['r(t+1)']).values)
    y_val_nn = np.asarray(window[1]['r(t+1)'].values)

    result_nn3 = compile_and_tune_model(model_3, param_nn, X_train_nn, y_train_nn, X_val_nn, y_val_nn)
    print('finished running')
    
    # get score
    result_nn3 = pd.DataFrame(result_nn3).sort_values('val_r2_score_wo_demeaning_nn',ascending=False)
    opt_score = result_nn3.iloc[0,1]
    print('opt_score: ', opt_score)
    opt_para = result_nn3.iloc[0,0]
    print('opt_para: ', opt_para)
    # results_nn1.append(result_nn1.iloc[0,])
    opt_scores_nn3.append(opt_score)
    opt_paras_nn3.append(opt_para)

start running
finished running
opt_score:  0.0020256787538528442
opt_para:  {'epoch': 50, 'learning_rate': 0.01, 'batch_size': 3000}
start running
finished running
opt_score:  0.00038105249404907227
opt_para:  {'epoch': 75, 'learning_rate': 0.01, 'batch_size': 3000}
start running
finished running
opt_score:  0.00035774335265159607
opt_para:  {'epoch': 100, 'learning_rate': 0.01, 'batch_size': 1500}
start running
finished running
opt_score:  0.00012042456364724785
opt_para:  {'epoch': 25, 'learning_rate': 0.01, 'batch_size': 3000}
start running
finished running
opt_score:  0.0003722429391928017
opt_para:  {'epoch': 25, 'learning_rate': 0.01, 'batch_size': 3000}
start running
finished running
opt_score:  0.0002454031491652131
opt_para:  {'epoch': 75, 'learning_rate': 0.025, 'batch_size': 3000}
start running
finished running
opt_score:  0.00023148830223362893
opt_para:  {'epoch': 50, 'learning_rate': 0.01, 'batch_size': 1500}
start running
finished running
opt_score:  0.000382712256396189

In [52]:
result_nn3

,params,val_r2_score_wo_demeaning_nn
8,"{'epoch': 25, 'learning_rate': 0.01, 'batch_si...",0.000383
14,"{'epoch': 50, 'learning_rate': 0.025, 'batch_s...",0.000371
26,"{'epoch': 75, 'learning_rate': 0.01, 'batch_si...",0.000357
29,"{'epoch': 100, 'learning_rate': 0.05, 'batch_s...",0.000349
5,"{'epoch': 25, 'learning_rate': 0.025, 'batch_s...",0.000345
17,"{'epoch': 50, 'learning_rate': 0.01, 'batch_si...",0.000340
24,"{'epoch': 75, 'learning_rate': 0.01, 'batch_si...",0.000338
6,"{'epoch': 25, 'learning_rate': 0.01, 'batch_si...",0.000335
33,"{'epoch': 100, 'learning_rate': 0.01, 'batch_s...",0.000334
34,"{'epoch': 100, 'learning_rate': 0.01, 'batch_s...",0.000318


best model

In [55]:
model_nn3 = Sequential([
            layers.Dense(layer1_n, activation='relu', input_dim=input_dim),
            layers.Dense(layer2_n, activation='relu'),
            layers.Dense(layer3_n, activation='relu'),
            layers.Dense(1, activation='linear')
        ])

last_epoch_metrics_nn3 = []
r2_nn3_tops = []
r2_nn3_bots = []

for i in range(8):
    print('start running')
    model_nn3.compile(optimizer=keras.optimizers.Adam(learning_rate=opt_para['learning_rate']),
                          loss='mean_squared_error', metrics=r2_score_wo_demeaning_nn)

    X_train_combined_nn = np.asarray(windows[i][3].drop(columns = ['r(t+1)']).values)
    y_train_combined_nn = np.asarray(windows[i][3]['r(t+1)'].values)
    
    X_test_nn = np.asarray(windows[i][2].drop(columns = ['r(t+1)']).values)
    y_test_nn = np.asarray(windows[i][2]['r(t+1)'].values)
    
    # fit the model
    history = model_nn3.fit(X_train_combined_nn, y_train_combined_nn, epochs=opt_paras_nn3[i]['epoch'], batch_size=opt_paras_nn3[i]['batch_size'],
                                validation_data=(X_test_nn, y_test_nn), verbose=0)
    print('finished')
    # get scores
    last_epoch_metric = history.history['val_r2_score_wo_demeaning_nn'][-1]
    last_epoch_metrics_nn3.append(last_epoch_metric)
    print('best score: ', last_epoch_metric)
    
    # top 100
    y_pred_nn3_top = model_nn3.predict(X_test_top)
    r2_nn3_top = r2_score_wo_demeaning(y_test_top, y_pred_nn3_top)
    r2_nn3_tops.append(r2_nn3_top)
    print('top best: ', r2_nn3_top)
    
    # bot 100
    y_pred_nn3_bot = model_nn3.predict(X_test_bot)
    r2_nn3_bot = r2_score_wo_demeaning(y_test_bot, y_pred_nn3_bot)
    r2_nn3_bots.append(r2_nn3_bot)
    print('bot best: ', r2_nn3_bot)

start running
finished
best score:  -0.03544444218277931
335/335 [==============================] - 1s 2ms/step
top best:  -0.03730309009552002
335/335 [==============================] - 1s 2ms/step
bot best:  -0.03580498695373535
start running
finished
best score:  -0.10656368732452393
335/335 [==============================] - 1s 2ms/step
top best:  -0.05102801322937012
335/335 [==============================] - 1s 2ms/step
bot best:  -0.13647222518920898
start running
finished
best score:  -0.10287187248468399
335/335 [==============================] - 1s 2ms/step
top best:  -0.06353926658630371
335/335 [==============================] - 1s 2ms/step
bot best:  -0.07618284225463867
start running
finished
best score:  -0.1627427190542221
335/335 [==============================] - 1s 2ms/step
top best:  -0.04085242748260498
335/335 [==============================] - 1s 3ms/step
bot best:  -0.11127936840057373
start running
finished
best score:  -0.12459839880466461
335/335 [===========

In [56]:
last_epoch_metrics_nn3

[-0.03544444218277931,
 -0.10656368732452393,
 -0.10287187248468399,
 -0.1627427190542221,
 -0.12459839880466461,
 -0.1097860336303711,
 -0.11114692687988281,
 -0.06862801313400269]

In [57]:
print("NN3 average R^2-oos:", sum(last_epoch_metrics_nn3) / len(last_epoch_metrics_nn3))

NN3 average R^2-oos: -0.10272276168689132


**top and bottom 100 prediction**

In [59]:
r2_nn3_tops

[-0.03730309009552002,
 -0.05102801322937012,
 -0.06353926658630371,
 -0.04085242748260498,
 -0.03298938274383545,
 -0.02205491065979004,
 -0.00018799304962158203,
 -0.002364635467529297]

In [60]:
r2_nn3_bots

[-0.03580498695373535,
 -0.13647222518920898,
 -0.07618284225463867,
 -0.11127936840057373,
 -0.06191408634185791,
 -0.01826465129852295,
 0.015552103519439697,
 0.062109529972076416]

In [62]:
print("Top 100 stocks R^2-oos:", sum(r2_nn3_tops) / len(r2_nn3_tops))

print("Bottom 100 stocks R^2-oos:", sum(r2_nn3_bots) / len(r2_nn3_bots))

Top 100 stocks R^2-oos: -0.0312899649143219
Bottom 100 stocks R^2-oos: -0.045282065868377686


In [ ]:
# result_nn3 = pd.DataFrame(result_nn3).sort_values('val_r2_score_wo_demeaning_nn',ascending=False)
# opt_score = result_nn3.iloc[0,1]
# opt_para = result_nn3.iloc[0,0]

In [ ]:
# result_nn3.iloc[0,1]

In [ ]:
# result_nn3.iloc[0,0]

In [ ]:
# model_nn3.compile(optimizer=keras.optimizers.Adam(learning_rate=opt_para['learning_rate']),
#                       loss='mean_squared_error', metrics=r2_score_wo_demeaning_nn)

# history = model_nn3.fit(X_train_combined_nn, y_train_combined_nn, epochs=opt_para['epoch'], batch_size=opt_para['batch_size'],
#                             validation_data=(X_test_nn, y_test_nn), verbose=0)
        
# last_epoch_metric = history.history['val_r2_score_wo_demeaning_nn'][-1]

